## Part One

In [11]:
from pathlib import Path
import cv2

# find repo root -> feature_detection
p = Path.cwd()
while p != p.parent and not (p / "feature_detection").exists():
    p = p.parent
fd = p / "feature_detection"

# paths
img_path = fd / "images" / "example-image.jpg"
out_path = fd / "images" / "example-image_sift_keypoints.png"

# load gray
gray = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)

# SIFT detect (defaults) and draw rich keypoints
sift = cv2.SIFT_create()
kps = sift.detect(gray, None)
out = cv2.drawKeypoints(gray, kps, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# save viz
cv2.imwrite(str(out_path), out)


True

## Part Two

In [12]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt

# Find feature_detection/ directory regardless of working directory
p = Path.cwd()
while p != p.parent and not (p / "feature_detection").exists():
    p = p.parent
fd = p / "feature_detection"

gray = cv2.imread(str(fd / "images" / "example-image.jpg"), cv2.IMREAD_GRAYSCALE)

# Compare a few tuned SIFT variants
variants = {
    "Default": cv2.SIFT_create(),
    "Higher contrastThreshold (0.1)": cv2.SIFT_create(contrastThreshold=0.1),
    "Lower edgeThreshold (5)": cv2.SIFT_create(edgeThreshold=5)
}

# Save the best (Default in this example) to images folder
best_sift = cv2.SIFT_create(edgeThreshold=5)
kps = best_sift.detect(gray, None)
out = cv2.drawKeypoints(gray, kps, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite(str(fd / "images" / "example-image_sift_tuned.png"), out)


True

## Part Three

In [ ]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Locate repo dir + load grayscale
p = Path.cwd()
while p != p.parent and not (p / "feature_detection").exists():
    p = p.parent
fd = p / "feature_detection"
img_path = fd / "images" / "example-image.jpg"

gray = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)

# Detect keypoints (tuned like Part 2) and compute descriptors
sift = cv2.SIFT_create(edgeThreshold=5)
kps = sift.detect(gray, None)
kps, desc = sift.compute(gray, kps)  

# Pick one keypoint (strongest response)
idx = int(np.argmax([kp.response for kp in kps]))
kp = kps[idx]
d = desc[idx]               # (128,)

rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
x, y = kp.pt
s = kp.size                 # diameter of the region
half = int(round(s/2))
x1, y1 = int(x - half), int(y - half)
x2, y2 = int(x + half), int(y + half)
cv2.circle(rgb, (int(x), int(y)), 3, (255, 0, 0), -1)
cv2.rectangle(rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Reshape 128-D descriptor to 4x4 cells × 8 bins
grid = d.reshape(4, 4, 8)

fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(rgb)
ax1.set_title("Keypoint & patch (scale from size)")
ax1.axis("off")

ax2 = fig.add_subplot(1, 2, 2)
# Build a small matrix summarizing the dominant bin per cell to keep it minimal
dom = np.argmax(grid, axis=2)   # 4x4 of [0..7]
im = ax2.imshow(dom, vmin=0, vmax=7)
ax2.set_title("Descriptor (4×4 cells, value = dominant orientation bin)")
ax2.set_xticks(range(4)); ax2.set_yticks(range(4))
plt.colorbar(im, ax=ax2, fraction=0.046, pad=0.04, ticks=range(8))

plt.tight_layout()

# Save final composite output
out_path = fd / "images" / "example-image_sift_descriptor.png"
fig.savefig(str(out_path), dpi=200, bbox_inches="tight")
plt.close(fig)


## Part Four

In [13]:
from pathlib import Path
import cv2
import numpy as np

p = Path.cwd()
while p != p.parent and not (p / "feature_detection").exists():
    p = p.parent
fd = p / "feature_detection"

# paths
img1_path = fd / "images" / "example-image.jpg"
img2_path = fd / "images" / "example-image-transformed.jpg"
matches_path = fd / "images" / "example-image_matches_top50.png"

# load original (gray for SIFT, color only for drawing later)
img1_gray = cv2.imread(str(img1_path), cv2.IMREAD_GRAYSCALE)
img1_bgr  = cv2.cvtColor(img1_gray, cv2.COLOR_GRAY2BGR)

# make a simple transformed copy of image1 (rotation + slight scale)
h, w = img1_gray.shape
M = cv2.getRotationMatrix2D((w//2, h//2), 20, 0.9)  # 20° rotate, 0.9 scale
img2_gray = cv2.warpAffine(img1_gray, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
img2_bgr  = cv2.cvtColor(img2_gray, cv2.COLOR_GRAY2BGR)
cv2.imwrite(str(img2_path), img2_gray)  # save the transformed image

sift = cv2.SIFT_create(edgeThreshold=5)
kp1, des1 = sift.detectAndCompute(img1_gray, None)
kp2, des2 = sift.detectAndCompute(img2_gray, None)

# brute-force match with L2, enforce 1-1 matches
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda m: m.distance)[:50]  # top 50 for clarity

# draw and save the match viz
matched = cv2.drawMatches(
    img1_bgr, kp1,
    img2_bgr, kp2,
    matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
cv2.imwrite(str(matches_path), matched)

True